## Import ##

In [ ]:
!pip install biodivine_aeon==1.3.0a3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 3.7 MB/s eta 0:00:00


In [ ]:
from biodivine_aeon import *
import requests
import itertools

## Functions ##

### Implementation Functions ###

In [ ]:
class EnrichmentBehaviourClass:
    def __init__(self, ) -> None:
        self.attractors = list()
        self.attractor_types = list()

    def add_attractor(self, attractor):
        self.attractors.append(attractor)
        self.attractor_types.append(attractor.attractor_type)

    def goterm_intersection(self):
        intersect = self.attractors[0].go_terms_set
        for attractor in self.attractors[1:]:
            intersect = intersect.intersection(attractor.go_terms_set)
        return intersect

    def goterm_unique(self):
        unique = []
        for attractor in self.attractors:
            unique_set = attractor.go_terms_set
            for attractor2 in self.attractors:
                if attractor == attractor2: continue
                unique_set = unique_set.difference(attractor2.go_terms_set)
            unique.append(unique_set)
        return unique

    def __str__(self):
        result = "Behaviour class \n"
        for attractor in self.attractors:
            result += " |-- " + str(attractor) + "\n"
        return result

    def __repr__(self):
        result = "Behaviour class \n"
        for attractor in self.attractors:
            result += " |-- " + str(attractor) + "\n"
        return result


class EnrichmentAttractor:
    def __init__(self, attractor_type, enrichment_result, fdr) -> None:
        self.fdr = fdr
        self.attractor_type = attractor_type
        self.goterms = dict()
        self.go_terms_set = set()

        if enrichment_result is None:
            self.mapped_ids = []
            self.unmapped_ids = []
            return

        self.mapped_ids = enrichment_result.mapped_ids
        self.unmapped_ids = enrichment_result.unmapped_ids

        for process in enrichment_result.result:
            go_term = EnrichmentGOterm(process)
            if go_term.fdr > self.fdr: continue
            self.goterms[go_term.go_id] = go_term
            self.go_terms_set.add(go_term.go_id)

    def get_goterms_by_set(self, wanted):
        return [self.goterms[go_id] for go_id in wanted if go_id in self.goterms]

    def get_all_goterms(self):
        return self.goterms

    def get_plus_goterms(self):
        return [goterm for goterm in self.goterms.values() if goterm.plus_minus == "+"]

    def get_minus_goterms(self):
        return [goterm for goterm in self.goterms.values() if goterm.plus_minus == "-"]

    def __str__(self):
        return f"{self.attractor_type}"

    def __repr__(self):
        return f"{self.attractor_type}"


class EnrichmentGOterm:
    def __init__(self, process) -> None:
        self.go_id = process.get("term", {}).get("id", "")
        self.process_name = process["term"]["label"]
        self.fold_enrichment = process["fold_enrichment"]
        self.fdr = process["fdr"]
        self.expected = process["expected"]
        self.number_in_reference = process["number_in_reference"]
        self.p_value = process["pValue"]
        self.plus_minus = process["plus_minus"]

    def __repr__(self):
        return f"{self.plus_minus}{self.process_name}"

    def __str__(self):
        return f"{self.plus_minus}{self.process_name}"


In [ ]:
class EnrichmentResult:
    def __init__(self, enrichmentData):
        self.data = enrichmentData

        self.input = self.data["results"]["input_list"]
        self.organism = self.input["organism"]
        self.mapped_ids = self.input["mapped_ids"]
        self.mapped_count = self.input["mapped_count"]
        self.unmapped_ids = self.input["unmapped_ids"]
        self.unmapped_count = self.input["unmapped_count"]
        self.result = self.data["results"]["result"]  # Sorted by FDR


def prepare_list_for_enrichment(nodes):
    as_string = str(nodes)[1:-1]
    as_string = as_string.replace("\'", "")
    return as_string


def prepare_enrichment_result(enrichment):
    if isinstance(enrichment, dict) and 'search' in enrichment and isinstance(enrichment['search'], dict) and 'error' in enrichment['search']:
        return None

    enrichment_result = EnrichmentResult(enrichment)
    return enrichment_result


def get_enrichment(genes_string, organism_id, goterm_type, test_type="FISHER", correction="FDR"):
    input_genes = genes_string
    organism = organism_id
    test_type = test_type   # FISHER, BINOMIAL
    correction = correction # FDR, BONFERRONI, NONE
    # refInputList  <- potential extension
    # refOrganism

    match goterm_type:
        case "MF": data_type = "GO:0003674"
        case "BP": data_type = "GO:0008150"
        case "CC": data_type = "GO:0005575"
        case _:
            print('Wrong goterm_type. Use "MF","BP","CC" instead. Ending get_enrichment function.')
            return

    req_link = f"https://pantherdb.org/services/oai/pantherdb/enrich/overrep?geneInputList={input_genes}&organism={organism}&annotDataSet={data_type}&enrichmentTestType={test_type}&correction={correction}"
    headers = {"Content-Type": "application/json"}
    response = requests.get(req_link, headers=headers)
    if response.status_code == 200:
        data = response.json()
        return data

    print("Failed to get data. Ending get_enrichment function.")
    return


In [ ]:
def make_union(attractors):
    unionized = attractors[0]
    for attractor in attractors[1:]:
        unionized = unionized.union(attractor)
    return unionized


def get_stability_percentage(node, stg, attractor):
    variable_on = stg.mk_subspace({node: True})

    on_in_attractor = attractor.intersect(variable_on).vertices().cardinality()
    off_in_attractor = attractor.minus(variable_on).vertices().cardinality()

    return round((on_in_attractor / (on_in_attractor + off_in_attractor)) * 100.0, 2)


def get_stabilities(classifiers, attractors, stg, calculate_unstable=True):
    all_dict = dict()
    unionized_attractors = make_union(attractors)
    for classifier in classifiers:
        subset = unionized_attractors.intersect_colors(classifiers[classifier])
        enclosing = subset.vertices().enclosing_named_subspace()
        unstable_variables = dict()

        if calculate_unstable:
            all_variables = set(stg.network_variable_names())
            stable_variables = set(enclosing)
            to_calculate = all_variables.difference(stable_variables)

            for variable in to_calculate:
                unstable_variables[variable] = get_stability_percentage(variable, stg, subset)

        stability_dict = dict()
        for node in enclosing:
            if enclosing[node] == True:
                stability_dict[node] = 100
            else:
                stability_dict[node] = 0
        if calculate_unstable:
            stability_dict = stability_dict | unstable_variables

        all_dict[classifier] = stability_dict
    return all_dict


def get_stable_nodes(classifiers, attractors, stg, lower_bound=0, upper_bound=100):
    calculate_unstable = True
    if lower_bound == 100 or upper_bound == 0:
        calculate_unstable = False

    stabilities = get_stabilities(classifiers, attractors, stg, calculate_unstable)

    result_dict = dict()
    for one_class in stabilities:
        print(one_class)
        print("  |", end='')

        sub = []
        for node in stabilities[one_class]:
            if stabilities[one_class][node] > upper_bound:
                continue
            if stabilities[one_class][node] < lower_bound:
                continue

            print(node + ": " + str(stabilities[one_class][node]) + "|", end='')
            sub.append(node)

        print()
        result_dict[one_class] = sub
    return result_dict


In [ ]:
def get_evaluated_nodes(phenotype, evaluation=True):
    result_list = []
    for node in phenotype:
        if node[0] == "+" and evaluation:
            result_list.append(node[1:])
        elif node[0] == "-" and not evaluation:
            result_list.append(node[1:])
    return result_list


### Extension Functions ###

In [ ]:
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
import os

def append_column_to_xlsx(filepath, data, column_name="Col"):
    """
    Adds data as a new column to an existing XLSX file.
    If the file does not exist, it will be created.
    """
    if os.path.exists(filepath):
        wb = load_workbook(filepath)
        ws = wb.active
    else:
        wb = Workbook()
        ws = wb.active

    if ws.max_column == 1 and ws.cell(row=1, column=1).value is None:
        col_index = 1
    else:
        col_index = ws.max_column + 1

    col_letter = get_column_letter(col_index)

    ws[f"{col_letter}1"] = column_name

    for i, value in enumerate(data, start=2):
        ws[f"{col_letter}{i}"] = value

    wb.save(filepath)

## Analysis ##

### Inicialization ###

In [ ]:
network = BooleanNetwork.from_file("mapk-TTTT.aeon")
human_id = "9606"

### Pipeline1 ###

Encapsulated algorithm described in the workflows folder.

In [ ]:
def pipeline1(behaviour_classes):
  ctx = SymbolicSpaceContext(network)
  stg = AsynchronousGraph(network, ctx)

  classification = Classification.classify_stable_phenotypes(ctx, stg)
  attractors = Attractors.attractors(stg)
  attractorClassifs = Classification.classify_attractor_bifurcation(stg, attractors)

  attractors_types = list(attractorClassifs)[0].feature_list()

  class_results = []
  for res in range(len(classification)):
    a = list(classification)[res]
    class_results.append(a.feature_list())

  write_combination = str(combination[0])[0] + str(combination[1])[0] + str(combination[2])[0] + str(combination[3])[0]
  new_behaviour_class = EnrichmentBehaviourClass()
  i = 0;

  for nodes, attractor_type in zip(class_results, attractors_types):
    to_enrich = get_evaluated_nodes(nodes)
    to_enrich = prepare_list_for_enrichment(to_enrich)
    enrichment = get_enrichment(to_enrich, human_id, "BP")
    enrichment_result = prepare_enrichment_result(enrichment)
    calculated_attractor = EnrichmentAttractor(attractor_type, enrichment_result, 0.05)
    new_behaviour_class.add_attractor(calculated_attractor)

    append_column_to_xlsx("mapk.xlsx", calculated_attractor.goterms, column_name=("["+ str(model_number) +"]" + "[" + write_combination+str(i) + "]"))
    i += 1

  behaviour_classes.append(new_behaviour_class)

The main body iterates over all evaluations of input nodes and stores the results in one list. The order of input nodes is important for future usage.

In [ ]:
combinations = list(itertools.product([True, False], repeat=4))
results = dict()
model_number = 0
behaviour_classes = []

for combination in combinations:
  print(model_number, "|", combination[0], combination[1], combination[2], combination[3])

  network.set_update_function("DNA_damage", str(combination[0]).lower())
  network.set_update_function("EGFR_stimulus", str(combination[1]).lower())
  network.set_update_function("FGFR3_stimulus", str(combination[2]).lower())
  network.set_update_function("TGFBR_stimulus", str(combination[3]).lower())

  pipeline1(behaviour_classes)
  model_number += 1

0 | True True True True
1 | True True True False
2 | True True False True
3 | True True False False
4 | True False True True
5 | True False True False
6 | True False False True
7 | True False False False
8 | False True True True
9 | False True True False
10 | False True False True
11 | False True False False
12 | False False True True
13 | False False True False
14 | False False False True
15 | False False False False


Which attractors belong under each of the possible behaviour classes. Behaviour classes are ordered, attractors are unordered as AEON.py returns them in an unordered manner.

In [ ]:
i = 0
for a in behaviour_classes:
  print(str(i) + " ", end="")
  print(a)
  i += 1

0 Behaviour class 
 |-- stability

1 Behaviour class 
 |-- disorder

2 Behaviour class 
 |-- stability

3 Behaviour class 
 |-- disorder

4 Behaviour class 
 |-- stability

5 Behaviour class 
 |-- disorder

6 Behaviour class 
 |-- stability

7 Behaviour class 
 |-- stability
 |-- stability

8 Behaviour class 
 |-- stability

9 Behaviour class 
 |-- disorder

10 Behaviour class 
 |-- stability

11 Behaviour class 
 |-- disorder

12 Behaviour class 
 |-- stability

13 Behaviour class 
 |-- disorder

14 Behaviour class 
 |-- stability

15 Behaviour class 
 |-- stability
 |-- stability



Which IDs were mapped for given attractor

In [ ]:
for behaviour_class in behaviour_classes:
  print("--")
  for attractor in behaviour_class.attractors:
    print(attractor.attractor_type)
    print(attractor.mapped_ids)

--
stability
GADD45,ATF2,JUN,AP1,DUSP1,MAX,PTEN,GAB1,MTK1,FOXO3,ELK1,p21,p53,PPP2CA,p14,p38,TAK1,RAF,MYC,ATM,GRB2,PDK1
--
disorder
GADD45,ATF2,JUN,AP1,DUSP1,MAX,PTEN,GAB1,MTK1,FOXO3,ELK1,p21,p53,PPP2CA,p14,p38,MYC,ATM,PDK1
--
stability
GADD45,ATF2,JUN,AP1,DUSP1,MAX,PTEN,GAB1,MTK1,FOXO3,ELK1,p21,p53,PPP2CA,p14,p38,TAK1,RAF,MYC,ATM,GRB2,PDK1
--
disorder
GADD45,ATF2,JUN,AP1,DUSP1,MAX,PTEN,GAB1,MTK1,FOXO3,ELK1,p21,p53,PPP2CA,p14,p38,MYC,ATM,PDK1
--
stability
GADD45,ATF2,JUN,AP1,DUSP1,MAX,PTEN,GAB1,MTK1,FOXO3,ELK1,p21,p53,PPP2CA,p14,p38,TAK1,RAF,MYC,ATM,GRB2,PDK1
--
disorder
GADD45,ATF2,JUN,AP1,DUSP1,MAX,PTEN,GAB1,MTK1,FOXO3,ELK1,p21,p53,PPP2CA,p14,p38,MYC,ATM,PDK1
--
stability
GADD45,ATF2,JUN,AP1,DUSP1,MAX,PTEN,GAB1,MTK1,FOXO3,ELK1,p21,p53,PPP2CA,p14,p38,TAK1,RAF,MYC,ATM,GRB2,PDK1
--
stability
GADD45,ATF2,JUN,AP1,DUSP1,MAX,PTEN,GAB1,MTK1,FOXO3,ELK1,p21,p53,PPP2CA,p14,p38,MYC,ATM,PDK1
stability
GADD45,ATF2,JUN,AP1,DUSP1,MAX,PTEN,MTK1,FOXO3,ELK1,p21,p53,PPP2CA,p14,p38,MYC,ATM
--
stability
GA

Classes made out of attractors that contain same GO Terms. Eclass containing no GO Terms is left out.

In [ ]:
Aclass = behaviour_classes[0].attractors[0].go_terms_set
Bclass = behaviour_classes[1].attractors[0].go_terms_set
Cclass = behaviour_classes[8].attractors[0].go_terms_set
Dclass = behaviour_classes[7].attractors[0].go_terms_set

How many GO Terms does each class contain.

In [ ]:
print("A: " + str(len(Aclass)))
print("B: " + str(len(Bclass)))
print("C: " + str(len(Cclass)))
print("D: " + str(len(Dclass)))

A: 334
B: 294
C: 292
D: 284


GO Terms specific only for Bclass.

In [ ]:
Bspecific = Bclass.difference(Aclass).difference(Cclass).difference(Dclass)
behaviour_classes[1].attractors[0].get_goterms_by_set(Bspecific)

[+regulation of striated muscle cell apoptotic process,
 +regulation of cardiac muscle cell apoptotic process,
 +positive regulation of cell population proliferation]

Mapped IDs by attractors from parametrisation TTTF and TFFF.  

In [ ]:
print(behaviour_classes[1].attractors[0].unmapped_ids)
print(behaviour_classes[7].attractors[1].unmapped_ids)

Apoptosis,CREB,DNA_damage,EGFR_stimulus,FGFR3_stimulus,Growth_Arrest,JNK,MSK,PI3K,TAOK
Apoptosis,CREB,DNA_damage,Growth_Arrest,JNK,MSK,PI3K,TAOK


 Visualisation of GO Terms can be done through https://2019.webgestalt.org/2017/GOView/

### Pipeline2 ###

In [ ]:
def pipeline2(behaviour_classes):

  ctx = SymbolicSpaceContext(network)
  stg = AsynchronousGraph(network, ctx)

  classification = Classification.classify_stable_phenotypes(ctx, stg)

  attractors = Attractors.attractors(stg)
  attractorClassifs = Classification.classify_attractor_bifurcation(stg, attractors)
  attractors_types = list(attractorClassifs)[0].feature_list()


  enrichment_input_a3 = get_stable_nodes(attractorClassifs, attractors, stg, 40, 60)
  enrichment_input_a3 = list(enrichment_input_a3.values())[0]
  behaviour_class_a3 = EnrichmentBehaviourClass()

  to_enrich = prepare_list_for_enrichment(enrichment_input_a3)
  enrichment = get_enrichment(to_enrich, "9606", "BP")
  enrichment_result = prepare_enrichment_result(enrichment)
  calculated_attractor = EnrichmentAttractor("CombinedAttractor", enrichment_result, 0.05)
  behaviour_class_a3.add_attractor(calculated_attractor)

  write_combination = str(combination[0])[0] + str(combination[1])[0] + str(combination[2])[0] + str(combination[3])[0]
  append_column_to_xlsx("mapkPipeline2.xlsx", calculated_attractor.goterms, column_name=("["+ str(i) +"]" + "[" + write_combination + "]"))


  behaviour_classes.append(behaviour_class_a3)

Pipeline2 prints the % of states in which the nodes were evaluated to 1. Same values may indicate relation between them. This pipeline is usefull for unstable attractors.

In [ ]:
combinations = list(itertools.product([True, False], repeat=4))
vysledky = dict()
i = 0
behaviour_classes2 = []

for combination in combinations:
  print(i, "|", combination[0], combination[1], combination[2], combination[3])

  network.set_update_function("DNA_damage", str(combination[0]).lower())
  network.set_update_function("EGFR_stimulus", str(combination[1]).lower())
  network.set_update_function("FGFR3_stimulus", str(combination[2]).lower())
  network.set_update_function("TGFBR_stimulus", str(combination[3]).lower())

  pipeline2(behaviour_classes2)
  i += 1


0 | True True True True
["stability"]
  |
1 | True True True False
["disorder"]
  |RAF: 50.0|EGFR: 47.06|SOS: 50.0|GRB2: 50.98|FGFR3: 47.06|MAP3K1_3: 50.0|FRS2: 49.02|RAS: 50.0|
2 | True True False True
["stability"]
  |
3 | True True False False
["disorder"]
  |RAF: 50.0|EGFR: 42.86|SOS: 50.0|PLCG: 57.14|GRB2: 50.0|PKC: 42.86|MAP3K1_3: 50.0|RAS: 50.0|
4 | True False True True
["stability"]
  |
5 | True False True False
["disorder"]
  |RAF: 50.0|PLCG: 55.56|SOS: 50.0|GRB2: 51.85|PKC: 44.44|FGFR3: 44.44|MAP3K1_3: 50.0|FRS2: 48.15|RAS: 50.0|
6 | True False False True
["stability"]
  |
7 | True False False False
["stability", "stability"]
  |PDK1: 50.0|GAB1: 50.0|PI3K: 50.0|
8 | False True True True
["stability"]
  |
9 | False True True False
["disorder"]
  |p21: 50.0|p70: 50.0|p53: 49.97|GADD45: 50.03|ERK: 50.03|MAX: 50.0|MAP3K1_3: 49.97|Growth_Arrest: 50.0|ATF2: 50.0|SOS: 49.97|AKT: 50.0|MSK: 50.0|FRS2: 49.02|RAS: 50.03|SPRY: 50.0|AP1: 50.0|RAF: 50.0|FOXO3: 50.0|MTK1: 49.97|JUN: 50.0|Ap

GO Terms were found for 6 parametrisations.

In [ ]:
for behaviour_class in behaviour_classes2:
  for a in behaviour_class.attractors:
    print(a.go_terms_set)


set()
{'GO:0048513', 'GO:0044344', 'GO:0009653', 'GO:0048731', 'GO:0042327', 'GO:0048732', 'GO:0038128', 'GO:1905207', 'GO:0050793', 'GO:0033135', 'GO:0048584', 'GO:0007166', 'GO:0007399', 'GO:0048009', 'GO:0045937', 'GO:0001932', 'GO:0043410', 'GO:0001934', 'GO:0007173', 'GO:0038127', 'GO:0007169', 'GO:0030154', 'GO:0048880', 'GO:1902531', 'GO:0000165', 'GO:0071363', 'GO:0008543', 'GO:0010562', 'GO:0001654', 'GO:0008286', 'GO:0007275', 'GO:1905208', 'GO:0071774', 'GO:1902533', 'GO:0033138', 'GO:0007167', 'GO:0048869', 'GO:0014044', 'GO:0009887', 'GO:0008283', 'GO:0031401', 'GO:0009719', 'GO:0035019', 'GO:0051093', 'GO:0043010', 'GO:0070848', 'GO:0022008', 'GO:0141124', 'GO:0150063', 'GO:0070372', 'GO:0043408', 'GO:0014037', 'GO:0071495'}
set()
{'GO:0048009', 'GO:0038127', 'GO:0033138', 'GO:0038128', 'GO:0033135'}
set()
{'GO:0048009', 'GO:0071774', 'GO:0044344', 'GO:0070848', 'GO:0007169', 'GO:0007167', 'GO:0071363', 'GO:0008543', 'GO:0043408', 'GO:0071495'}
set()
set()
set()
{'GO:0071

Intersecting GO Terms among the parametrisations.

In [ ]:
intersect = behaviour_classes2[9].attractors[0].go_terms_set
intersect = intersect.intersection(behaviour_classes2[11].attractors[0].go_terms_set)
intersect = intersect.intersection(behaviour_classes2[13].attractors[0].go_terms_set)
intersect = intersect.intersection(behaviour_classes2[1].attractors[0].go_terms_set)

In [ ]:
behaviour_classes2[9].attractors[0].get_goterms_by_set(intersect)

[+animal organ development,
 +anatomical structure morphogenesis,
 +system development,
 +positive regulation of phosphorylation,
 +gland development,
 +ERBB2 signaling pathway,
 +regulation of developmental process,
 +positive regulation of response to stimulus,
 +regulation of peptidyl-serine phosphorylation,
 +nervous system development,
 +cell surface receptor signaling pathway,
 +insulin-like growth factor receptor signaling pathway,
 +positive regulation of phosphate metabolic process,
 +regulation of protein phosphorylation,
 +positive regulation of MAPK cascade,
 +positive regulation of protein phosphorylation,
 +sensory system development,
 +ERBB signaling pathway,
 +cell surface receptor protein tyrosine kinase signaling pathway,
 +cell differentiation,
 +regulation of intracellular signal transduction,
 +MAPK cascade,
 +cellular response to growth factor stimulus,
 +positive regulation of phosphorus metabolic process,
 +eye development,
 +multicellular organism development,


In [ ]:
intersect = intersect.intersection(behaviour_classes2[5].attractors[0].go_terms_set)
behaviour_classes2[9].attractors[0].get_goterms_by_set(intersect)

[+insulin-like growth factor receptor signaling pathway,
 +response to growth factor,
 +cell surface receptor protein tyrosine kinase signaling pathway,
 +enzyme-linked receptor protein signaling pathway,
 +cellular response to growth factor stimulus,
 +regulation of MAPK cascade,
 +cellular response to endogenous stimulus]

In [ ]:
intersect = intersect.intersection(behaviour_classes2[3].attractors[0].go_terms_set)
behaviour_classes2[9].attractors[0].get_goterms_by_set(intersect)

[+insulin-like growth factor receptor signaling pathway]